In [1]:
import regex as re
from pathlib import Path
import spacy
from spacy import displacy
import pandas as pd
import numpy as np
import coreferee
from sentence_transformers import SentenceTransformer, util

c:\Users\zisak\anaconda3\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
import transformers
print(transformers.__version__)

4.36.0


### Old Resolving

In [3]:

def resolved(nlp, text, questions_mlhash_lazma=[], title=""):
    flag = True

    official_subject = title if title else "Unknown"

    sentences = []
    prev_subjs = [title]

    text = nlp(text)

    for sent in text.sents:
        prev_subj, compound_is, last_word = "", "", ""

        dep_word = [word.dep_ for word in sent]     # hyraga3 kol kelma no3ha eih (subject, adjective, kda y3ny)
        word_dep_count_subj = [dep_word.index(word) for word in dep_word if word in ('nsubj', 'subj', 'nsubjpass')]

        try:
            word_dep_count_subj = word_dep_count_subj[-1] + 1
        except IndexError:
            word_dep_count_subj = 1

        more_subjs = [word for word in dep_word if word in ('nsubj', 'subj', 'nsubjpass')]
        for word in sent:
            if len(more_subjs) > 1:
                if word.dep_ in more_subjs:
                    if word.dep_ in ['nsubjpass']:
                        break
                    elif word.dep_ in ('nsubj','subj'):
                        if word_dep_count_subj > 0:
                            if word.dep_ in ('compound') or word.dep_ in ('nmod', 'amod'):
                                if compound_is == "":
                                    compound_is = str(word)
                                    word_dep_count_subj = word_dep_count_subj - 1
                                else:
                                    compound_is = compound_is+ " " +str(word)
                                    word_dep_count_subj = word_dep_count_subj - 1

                            elif word.dep_ in ('nsubj', 'subj', 'nsubjpass'):
                                pronoun = [i for i in word.subtree]

                                if compound_is == "":
                                    if str(word) not in ('he', 'she', 'it'):
                                        prev_subj = str(word)
                                        if str(pronoun[0]) not in ('his','her', 'its'):
                                            prev_subjs = [prev_subj]
                                            official_subject = prev_subjs[0]
                                            word_dep_count_subj = word_dep_count_subj - 1

                                else:
                                    if str('poss') in [str(i.dep_) for i in word.subtree]:
                                        prev_subj = compound_is
                                        word_dep_count_subj = word_dep_count_subj - 1
                                        prev_subjs = [prev_subj]
                                    else:
                                        prev_subj = compound_is+" "+str(word)
                                        word_dep_count_subj = word_dep_count_subj - 1
                                        prev_subjs = [prev_subj]
                                        official_subject = prev_subjs[0]

                                if str(word) in ('he', 'she', 'it'):
                                    new_word = prev_subjs[-1]
                                    sentences.append(re.sub(r'\b' + str(word) + r'\b', str(new_word), str(sent)))
                                    flag = False

                                if pronoun and str(pronoun[0]) in ('his','her', 'its'):
                                    new_word = str(official_subject)+"\'s"
                                    sentences.append(re.sub(r'\b' + str(pronoun[0]) + r'\b', str(new_word), str(sent)))
                                    flag = False


                            elif word.dep_ in ('nsubj','subj','nsubjpass') and str(word) not in ('he', 'she', 'it'):
                                last_word = word
                            else:
                                pass
            else:
                if word_dep_count_subj > 0:
                    if word.dep_ in ('compound') or word.dep_ in ('nmod', 'amod'):
                        if compound_is == "":       # dyh awl kelma fl compound kda
                            compound_is = str(word)
                            word_dep_count_subj = word_dep_count_subj - 1
                        else:   # msh awl kelma, fa zawedha 3l mwgood w 5las
                            compound_is = compound_is+ " " +str(word)
                            word_dep_count_subj = word_dep_count_subj - 1

                    elif word.dep_ in ('nsubj', 'subj', 'nsubjpass'):
                        pronoun = [i for i in word.subtree]

                        if compound_is == "":
                            if str(word) not in ('he', 'she', 'it'):
                                prev_subj = str(word)
                                if str(pronoun[0]) not in ('his','her', 'its'):
                                    prev_subjs = [prev_subj]
                                    official_subject = prev_subjs[0]        # dh el subject bta3 el gomal el b3deeha y3ny
                                    word_dep_count_subj = word_dep_count_subj - 1

                        else:
                            if str('poss') in [str(i.dep_) for i in word.subtree]:  # poss: his, her, kda y3ny
                                prev_subj = compound_is
                                word_dep_count_subj = word_dep_count_subj - 1
                                prev_subjs = [prev_subj]
                            else:
                                prev_subj = compound_is+" "+str(word)
                                word_dep_count_subj = word_dep_count_subj - 1
                                prev_subjs = [prev_subj]
                                official_subject = prev_subjs[0]

                        if str(word) in ('he', 'she', 'it'):
                            new_word = prev_subjs[-1]
                            sentences.append(re.sub(r'\b' + str(word) + r'\b', str(new_word), str(sent)))
                            flag = False    # 5ly el flag false 3lshan my3mlsh append tany ta7t

                        if pronoun and str(pronoun[0]) in ('his','her', 'its'):           # hysheel his w y7ot Leonardo DiCaprio's
                            new_word = str(official_subject)+"\'s"
                            sentences.append(re.sub(r'\b' + str(pronoun[0]) + r'\b', str(new_word), str(sent)))
                            flag = False

                    elif word.dep_ in ('nsubj','subj','nsubjpass') and str(word) not in ('he', 'she', 'it'):
                        last_word = word
                    else:
                        pass


        if flag:
            sentences.append(str(sent))     # 3mlnalha append fy 7eta tanya, fa msh hn3ml hena tany
        else:
            flag = True

    resolved_text = " ".join(sentences)
    return resolved_text

In [4]:
# text = """
# Lionel Andrés "Leo" Messi was born in 24 June 1987 is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team.
# Widely regarded as one of the greatest players of all time, Messi has won a record eight Ballon d'Or awards, a record six European Golden Shoes, and was named the world's best player for a record eight times by FIFA.
# Until 2021, he had spent his entire professional career with Barcelona, where he won a club-record 34 trophies, including ten La Liga titles, seven Copa del Rey titles, and the UEFA Champions League four times.
# With his country, he won the 2021 Copa América and the 2022 FIFA World Cup. A prolific goalscorer and creative playmaker, Messi holds the records for most goals (474), hat-tricks (36), and assists in La Liga (192). He has the most international goals by a South American male (106). Messi has scored over 800 senior career goals for club and country, and the most goals for a single club.
# """
# # change text to lowercase
# text = text.lower()
# resolved_text = resolved(nlp, text, title="Alice")
# resolved_text

This is the universal dependency for English: https://universaldependencies.org/en/dep/index.html

### Loading Model

In [5]:
nlp = spacy.load('en_core_web_md')
nlp.add_pipe("merge_entities")
nlp.add_pipe("merge_noun_chunks")
nlp.add_pipe('coreferee')
model = SentenceTransformer("all-MiniLM-L6-v2")

doc = """
Lionel Andrés "Leo" Messi was born in 24 June 1987 is an Argentine professional footballer plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team.
He played in Barcelona in 2010.
Widely regarded as one of the greatest players of all time, Messi has won a record eight Ballon d'Or awards, a record six European Golden Shoes, and was named the world's best player for a record eight times by FIFA.
Until 2021, he had spent his entire professional career with Barcelona, where he won a club-record 34 trophies, including ten La Liga titles, seven Copa del Rey titles, and the UEFA Champions League four times.
With his country, he won the 2021 Copa América and the 2022 FIFA World Cup. A prolific goalscorer and creative playmaker, Messi holds the records for most goals, hat-tricks, and assists in La Liga. He has the most international goals by a South American male. Messi has scored over 800 senior career goals for club and country, and the most goals for a single club.
"""
# doc = "Farah took a trophy. Farah win a cake."

c:\Users\zisak\anaconda3\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\zisak\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\zisak\anaconda3\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\zisak\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want

### resolve coreferences

In [6]:

def resolve_coreference(text):
    doc = nlp(text)
    doc_list = list(doc)
    doc._.coref_chains.print()
    resolving_indecies = []
    for _,item in enumerate(doc._.coref_chains):
        resolving_indecies.extend(item)
        
    for word in resolving_indecies:
        new_word = ""
        for index in word:
            if doc[index]._.coref_chains.resolve(doc[index]) is not None:
                temp = []
                for item in doc._.coref_chains.resolve(doc[index]):
                    temp.append(str(item))
                new_word = ", ".join(temp)
            
                doc_list[index] = new_word

    final_doc = []
    for item in doc_list:
        final_doc.append(str(item))
    return " ".join(final_doc)
        

### Preprocess the document

In [7]:
def preprocess_context(doc):
    text = doc.strip()
    text.replace(".", ",")
    resolved_text = resolve_coreference(text)
    resolved_text = resolved_text.strip()
    resolved_text = resolved_text.replace("  ", " ").replace(" ,", ",").replace(" .", ".").replace("\n", "")
    return resolved_text

In [8]:
resolved_doc = preprocess_context(doc)
cleaned_doc = nlp(resolved_doc)
sentences = [one_sentence.text.strip() for one_sentence in cleaned_doc.sents]

0: Lionel Andrés "Leo" Messi(0), He(18), Messi(35), he(57), he(65), he(83), Messi(96), He(109), Messi(115)
1: Barcelona(21), Barcelona(62)


In [9]:
excludesPerQuestionType = {
    "when": "Times",
    "where": "Locations",
    "who": "Subject",
    "what": "Objects",
    "how": "States"
}

In [10]:
question = "when was messi defeated by Realmadrid?" 
answer = "Ziad said that Farah went to the mall with Boody."
ans_nlp = nlp(answer)
question_nlp = nlp(question)
question_type = question_nlp[0].text.lower()
# what, when, where, who , --> how, how many

In [11]:
for token in question_nlp:
    
    print(token.text, '-',token.pos_,'-', token.dep_,'-', token.ent_type_)
        

# displacy.render(question_nlp, style="dep")

when - SCONJ - advmod - 
was - AUX - auxpass - 
messi - NOUN - nsubjpass - 
defeated - VERB - ROOT - 
by - ADP - agent - 
Realmadrid - PROPN - pobj - PERSON
? - PUNCT - punct - 


In [12]:
for token in ans_nlp:
    
    print(token.text, '-',token.pos_,'-', token.dep_,'-', token.ent_type_)
        

# displacy.render(ans_nlp, style="dep")

Ziad - PROPN - nsubj - PERSON
said - VERB - ROOT - 
that - SCONJ - mark - 
Farah - PROPN - nsubj - PERSON
went - VERB - ccomp - 
to - ADP - prep - 
the mall - NOUN - pobj - 
with - ADP - prep - 
Boody - PROPN - pobj - PERSON
. - PUNCT - punct - 


### Extracting the entities

In [13]:
def extract_subjects(sentence):
    subjects = []
    for token in sentence:
        if token.dep_ in ("nsubj","csubj", "nsubjpass"):
            if token.dep_ == "nsubjpass":
                verb = token.head
                for child in verb.children:
                    if child.dep_ == "agent":
                        subject = [str(t) for t in list(child.children)[0].subtree]
                        subject = " ".join(subject)
                        break
                    else:
                        subject = "Unknown"
                subjects.append((subject, verb))
            else:                       
                subtree_tokens = [str(t) for t in token.subtree]
                verb = token.head
                subjects.append((" ".join(subtree_tokens), verb))
    return subjects

def extract_objects(sentence):
    objects = []
    for token in sentence:
        if token.dep_ in ("dobj", "dative", "attr", "oprd", "acomp","ccomp", "xcomp", "nsubjpass"):
            subtree_tokens = [str(t) for t in token.subtree]
            verb = token.head
            objects.append((" ".join(subtree_tokens), verb))
    return objects

def extract_state(sentence):
    states = []
    for token in sentence:
        if token.pos_ =="VERB" or token.pos_ == "AUX":
            for child in token.children:
                if child.dep_ == "prep":
                    subtree_tokens = [str(t) for t in child.subtree]
                    states.append(((" ".join(subtree_tokens), token)))
    return states

def extract_time(sentence):
    times = {}
    for token in sentence:
        if token.pos_ == "VERB" or token.pos_ == "AUX":
            for child in token.subtree:
                if child.ent_type_ == "DATE" or child.ent_type_ == "TIME":
                    times[child.text] = token
    return list(times.items())

def extract_location(sentence):
    locations = {}
    for token in sentence:
        if token.pos_ == "VERB" or token.pos_ == "AUX":
            for child in token.subtree:
                if child.ent_type_ in ("GPE", "LOC", "FAC"):
                    locations[child.text] = token
    return list(locations.items())
                    

def extract_facts(sentence):
    sentence = nlp(sentence)
    states = extract_state(sentence)
    subjects = extract_subjects(sentence)
    objects = extract_objects(sentence)
    times = extract_time(sentence)
    locations = extract_location(sentence)
    print(subjects, objects)
    
    facts = pd.DataFrame(columns=["Subject", "Relation", "Objects", "States", "Times", "Locations"])
    
    for subject in subjects:
        verb = subject[1].lemma_
        currentSubject = subject[0]
        if verb in facts["Relation"].values:
            facts.loc[facts["Relation"] == verb, "Subject"] = currentSubject
        else:
            new_row = pd.DataFrame([{"Subject": currentSubject, "Relation": verb, "Objects": [], "States": [], "Times": [], "Locations": []}])
            facts = pd.concat([facts, new_row], ignore_index=True)
    for obj in objects:
        verb = obj[1].lemma_
        currentObj = obj[0]
        if verb in facts["Relation"].values:
            oldObjects = list(facts.loc[facts["Relation"] == verb, "Objects"].values[0])
            oldObjects.append(currentObj)
            facts.loc[facts["Relation"] == verb, "Objects"] = [oldObjects] 
            

    for state in states:
        verb = state[1].lemma_
        currentState = state[0]
        if verb in facts["Relation"].values:
            oldStates = list(facts.loc[facts["Relation"] == verb, "States"].values[0])
            oldStates.append(currentState)
            facts.loc[facts["Relation"] == verb, "States"] = [oldStates]
            
    for time in times:
        verb = time[1].lemma_
        currentTime = time[0]
        if verb in facts["Relation"].values:
            oldTimes = list(facts.loc[facts["Relation"] == verb, "Times"].values[0])
            oldTimes.append(currentTime)
            facts.loc[facts["Relation"] == verb, "Times"] = [oldTimes]
            
    for location in locations:
        verb = location[1].lemma_
        currentLocation = location[0]
        if verb in facts["Relation"].values:
            oldLocations = list(facts.loc[facts["Relation"] == verb, "Locations"].values[0])
            oldLocations.append(currentLocation)
            facts.loc[facts["Relation"] == verb, "Locations"] = [oldLocations]
            
    return facts

In [14]:
questionDF = extract_facts(question)

[('Realmadrid', defeated)] [('messi', defeated)]


In [15]:
def join_sentences_facts(sentences):
    all_facts = pd.DataFrame(columns=["Subject", "Relation", "Objects", "States", "Times", "Locations"])
    for sentence in sentences:
        facts = extract_facts(sentence)
        all_facts = pd.concat([all_facts, facts])
    all_facts = all_facts.groupby(["Subject", "Relation"], as_index=False).agg({
        "Objects": lambda x: [item for sublist in x for item in sublist],
        "States": lambda x: [item for sublist in x for item in sublist],
        "Times": lambda x: [item for sublist in x for item in sublist],
        "Locations": lambda x: [item for sublist in x for item in sublist]
    })
    return all_facts

In [16]:
factsDF = join_sentences_facts(sentences)

[('Unknown', born), ('an Argentine professional footballer', plays)] [('Lionel Andrés "Leo" Messi', born), ('an Argentine professional footballer plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team', is)]
[('Lionel Andrés "Leo" Messi', played)] []
[('Lionel Andrés "Leo" Messi', won)] [("a record eight Ballon d'Or awards , a record six European Golden Shoes", won), ("the world's best player", named)]
[('Lionel Andrés "Leo" Messi', spent), ('Lionel Andrés "Leo" Messi', won)] [('his entire professional career', spent), ('a club-record 34 trophies', won)]
[('Lionel Andrés "Leo" Messi', won)] [('the 2021 Copa América and the 2022 FIFA World Cup', won)]
[('Lionel Andrés "Leo" Messi', holds)] [('the records for most goals , hat-tricks', holds)]
[('Lionel Andrés "Leo" Messi', has)] [('the most international goals by a South American male', has)]
[('Lionel Andrés "Leo" Messi', scored)] [('over 800 senior career goals for club and country

In [17]:
# iterate the dataframe and change the subject to [Subject] and the relation to [Relation]
def change_subject_relation(factsDF):
    for index, row in factsDF.iterrows():
        factsDF.loc[index, "Subject"] = [row['Subject']]
        factsDF.loc[index, "Relation"] = [row['Relation']]
    return factsDF

In [18]:
newFactsDF = change_subject_relation(factsDF)
newFactsDF

,Subject,Relation,Objects,States,Times,Locations
0,"[Lionel Andrés ""Leo"" Messi]",[have],[the most international goals by a South Ameri...,[],[],[]
1,"[Lionel Andrés ""Leo"" Messi]",[hold],"[the records for most goals , hat-tricks]",[],[],[La Liga]
2,"[Lionel Andrés ""Leo"" Messi]",[play],[],"[in Barcelona, in 2010]",[2010],[Barcelona]
3,"[Lionel Andrés ""Leo"" Messi]",[score],[over 800 senior career goals for club and cou...,[],[],[]
4,"[Lionel Andrés ""Leo"" Messi]",[spend],[his entire professional career],"[Until 2021, with Barcelona , where Lionel And...",[2021],[Barcelona]
5,"[Lionel Andrés ""Leo"" Messi]",[win],"[a record eight Ballon d'Or awards , a record ...","[including ten La Liga titles , seven Copa del...",[],[]
6,[Unknown],[bear],"[Lionel Andrés ""Leo"" Messi]",[in 24 June 1987],[24 June 1987],[]
7,[an Argentine professional footballer],[play],[],"[as a forward, for]",[],[]


In [19]:
newQuestionDF = change_subject_relation(questionDF)
newQuestionDF

,Subject,Relation,Objects,States,Times,Locations
0,[Realmadrid],[defeat],[messi],[],[],[]


In [20]:
def similarity(factRow, questionRow, column):
    if len(factRow[column]) == 0 or len(questionRow[column]) == 0:
        return 0
    columnString = " ".join(factRow[column])
    questionString = " ".join(questionRow[column])
    embeddingFact = model.encode(columnString)
    embeddingQuestion = model.encode(questionString)
    return util.cos_sim(embeddingFact, embeddingQuestion)

In [21]:
def cost_function(factsDf, questionFact, excludeColumns=[]):
    cost = 0
    maxFactIdx = 0
    columnNames = ["Subject","Relation", "Objects", "States", "Times", "Locations"]
    for column in excludeColumns:
        columnNames.remove(column)
    for factIdx, factRow in factsDf.iterrows():
        currCost = 0
        for _, questionRow in questionFact.iterrows():
            for column in columnNames:
                currCost += similarity(factRow, questionRow, column)
        if currCost > cost:
            cost = currCost
            maxFactIdx = factIdx
    return maxFactIdx, cost.item()/len(columnNames)
        

In [22]:
correctIdx, _ = cost_function(factsDF, questionDF, excludeColumns=[excludesPerQuestionType[question_type]])
WhenAnswer = factsDF.loc[correctIdx, excludesPerQuestionType[question_type]]
" ".join(WhenAnswer)

'24 June 1987'

In [23]:
text = 'Lionel Andrés Leo Messi was born in 24 June 1987 is an Argentine professional footballer plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team.'
nlp_md = spacy.load("en_core_web_md")
nlp_md.add_pipe("merge_entities")
nlp_md.add_pipe("merge_noun_chunks")
doc_md = nlp_md(text)

In [24]:
messi1 = "Lionel Andrés Leo Messi"
messi2 = "Vegetables"
# to lower case
messi1 = messi1.lower()
messi2 = messi2.lower()
nlp_messi1 = nlp_md(messi1)
nlp_messi2 = nlp_md(messi2)

In [25]:
# make cosine similarity between the two vectors and normailze it
similarity = nlp_messi1.vector @ nlp_messi2.vector / (np.linalg.norm(nlp_messi1.vector) * np.linalg.norm(nlp_messi2.vector))
similarity

0.0102947485

In [26]:
model = SentenceTransformer("all-MiniLM-L6-v2")

c:\Users\zisak\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [27]:
emb1 = model.encode("win")
emb2 = model.encode("win")

cos_sim = util.cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim.item())

Cosine-Similarity: 1.0000001192092896


In [28]:
for token in doc_md:
    print(token.text)

Lionel Andrés Leo Messi
was
born
in
24 June 1987
is
an Argentine professional footballer
plays
as
a forward
for
and
captains
both Major League Soccer club Inter Miami
and
the Argentina national team
.


In [29]:
nlp = spacy.load("en_core_web_sm")
nlp.get_pipe("ner").labels

('CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART')